<a href="https://colab.research.google.com/github/tanyaryabov/ML/blob/master/Hybrid_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Content based recommendations. movie genres and searching for similar movies. To find the similarity between the two movies we will be using the cosine similarity metric.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive

Mounted at /content/drive
MyDrive  Shareddrives


In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#Load Dataset
movies = pd.read_csv('/content/drive/MyDrive/movies.csv')
movies['title'] = movies['title'].str.strip().str[:-7]
movies['genres'] = movies['genres'].str.replace('|', ' ')
movies.head()

,movieId,title,genres
0,1,Toy Story,Adventure Animation Children Comedy Fantasy
1,2,Jumanji,Adventure Children Fantasy
2,3,Grumpier Old Men,Comedy Romance
3,4,Waiting to Exhale,Comedy Drama Romance
4,5,Father of the Bride Part II,Comedy


In [5]:
ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv')
ratings = ratings.drop(columns=['timestamp'])

In [33]:
movies

,movieId,title,genres
0,1,Toy Story,"[0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,2,Jumanji,"[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,3,Grumpier Old Men,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,Waiting to Exhale,"[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,5,Father of the Bride Part II,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,"[1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
9738,193583,No Game No Life: Zero,"[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
9739,193585,Flint,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
9740,193587,Bungo Stray Dogs: Dead Apple,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(movies['genres'].values)
feature_names = vectorizer.get_feature_names()

In [7]:
genres = pd.DataFrame(x.toarray(), columns=feature_names)
genres['combined']= genres.values.tolist()
movies['genres'] = genres['combined']
movies.head()

,movieId,title,genres
0,1,Toy Story,"[0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,2,Jumanji,"[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,3,Grumpier Old Men,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,Waiting to Exhale,"[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,5,Father of the Bride Part II,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
def get_cossim(movieid, top):
    # Creating dataframe with only IDs and genres
    movies_to_search = movies[['movieId', 'genres']]
    # Remove the ID of the movie we are measuring distance to
    movies_to_search = movies_to_search[movies_to_search.movieId != movieid]
    # Saving distances to new column
    movies_to_search['dist'] = movies_to_search['genres'].apply(lambda x: cosine_similarity(np.array(x).reshape(1, -1), np.array(movies.loc[movies['movieId'] == movieid]['genres'].values[0]).reshape(1, -1)))
    # Remove the genres column
    movies_to_search = movies_to_search.drop(columns=['genres'])
    # Distance value is in the list inside of the list so we need to unpack it
    movies_to_search = movies_to_search.explode('dist').explode('dist')
    # Sort the data and return top values
    return movies_to_search.sort_values(by=['dist'], ascending=False)['movieId'].head(top).values

In [19]:
get_cossim(47, 5)

array([ 4689,  5712,  2204, 27773,  4226])

In [10]:
#function takes 10 top-rated movies by a selected user and returns 5 similar movies for each of those
def get_similar(userid):
    # Take all the movies watched by user
    movies_watched_by_user = ratings[ratings.userId == userid]
    movies_watched_by_user = movies_watched_by_user[movies_watched_by_user['rating'] > 4.5]  # Only 4.5 or higher rating filtered
    # Taking top 20 with highest ratings
    top_movies_user = (movies_watched_by_user.sort_values(by="rating", ascending=False).head(10))
    top_movies_user['watched_movieId'] = top_movies_user['movieId']
    top_movies_user = top_movies_user[['userId', 'watched_movieId']]
    # Find 5 similar movies for each of the selected above
    top_movies_user['similar'] = top_movies_user['watched_movieId'].apply(lambda x: (get_cossim(x, 5)))
    # Remove movies that user have already watched from recommendations
    result = [x for x in np.concatenate(top_movies_user['similar'].values, axis=0).tolist() if x not in top_movies_user.watched_movieId.values.tolist()]
    return result

In [48]:
#'get_top' function returns top N recommended movies sorted by mean user rating. (only movies with 10 or more ratings are used).
def get_top(userid, top):
    # taking movies that user may like
    smlr = get_similar(userid)    
    # Calculating mean rationg for every movie
    movie_data = pd.merge(ratings, movies, on='movieId')
    ratings_mean_count = pd.DataFrame(movie_data.groupby('movieId')['rating'].mean())
    ratings_mean_count['rating_counts'] = pd.DataFrame(movie_data.groupby('movieId')['rating'].count())
    # Sorting movies with 10 or more ratings by users
    ratings_mean_count = ratings_mean_count[ratings_mean_count['rating_counts'] > 10]
    # Returning top top= 10 movies sorted by rating
    ratings_mean_count[ratings_mean_count.index.isin(smlr)].sort_values(by=['rating'], ascending=False).head(top)
    final_rec= pd.merge(temp, movies, on= 'movieId')
    final_recommendation= final_rec.drop(columns= ['genres'])
    return final_recommendation

In [49]:
get_top(1, 5)

,movieId,rating,rating_counts,title
0,4011,4.155914,93,Snatch
1,4226,4.122642,159,Memento
2,1086,4.100000,25,Dial M for Murder
3,27773,4.089744,39,Old Boy
4,3328,4.052632,19,Ghost Dog: The Way of the Samurai
5,2944,3.968750,32,"Dirty Dozen, The"
6,1211,3.884615,13,"Wings of Desire (Himmel über Berlin, Der)"
7,37741,3.869565,23,Capote
8,21,3.494382,89,Get Shorty
9,41566,3.443548,62,"Chronicles of Narnia: The Lion, the Witch and ..."
